In [ ]:
import pandas as pd
import RNA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter, defaultdict
from scipy.interpolate import interp1d
from matplotlib import gridspec
from scipy.stats import spearmanr
import math
import random

### Import of Data

In [ ]:
#Import of T-RIP transcripts
df=pd.read_excel("C:/.../Enriched transcripts.xlsx", header=[1])

In [ ]:
#Import of Half Life Data for appropriate plots
df2=pd.read_excel("C:/../pnas.1712312115.sd01.xlsx", sheet_name="S2-Modeling Results", skiprows=[0,1,2,3])

In [ ]:
dic=defaultdict(str)
with open(r"C:/.../Araport11_5_utr_20220504") as f:
    line="1"
    while line:
        line=f.readline().strip()
        if ">" in line:
            name=line[1:line.index("|")-1]
            continue
        dic[name]+=line

In [ ]:
dic_3=defaultdict(str)
with open(r"C:/.../Araport11_3_utr_20220504") as f:
    line="1"
    while line:
        line=f.readline().strip()
        if ">" in line:
            name=line[1:line.index("|")-1]
            continue
        dic_3[name]+=line

### Metric Calculation

In [ ]:
Half_Life=dict(zip(df2['Unnamed: 0'].to_list(), list(map(lambda x: np.log(2)/x,df2["alpha_WT"].to_list()))))

In [ ]:
#Either mock_enriched or heat_enriched may be used for binning
#Change accordingly
mock_enriched=df[df["Log ratio mock"]>1]["geneID"].to_list()
heat_enriched=df[df["Log ratio heat"]>1]["geneID"].to_list()

In [ ]:
def MFE_random(i):
    model=RNA.md()
    model.temperature=22
    seq="".join([random.choices(["A","C","G","T"],weights=[0.25,0.25,0.25,0.25])[0] for i in range(i)])
    return RNA.fold_compound(seq,model).mfe()[1]

In [ ]:
rands=defaultdict(list)
for num in np.arange(40,604,4):
    for i in range(2000):
        rands[num]+=[MFE_random(num)]

In [ ]:
predicted_mfes={int(k):np.mean(v)for k,v in rands.items()}

interpolation=interp1d(list(predicted_mfes.keys()), list(predicted_mfes.values()))

reference_mfes={int(i):float(interpolation(i)) for i in np.arange(40,601,1)}

In [ ]:
class seq:
    def __init__(self, s):
        self.s=s
        self.model=RNA.md()
        self.model.temperature=22
        self.comp=RNA.fold_compound(self.s,self.model).mfe()
    
    pred_mfe=reference_mfes
    def __len__(self):
        return len(self.s)
    
    def structure2D(self):
        return RNA.db_to_tree_string(self.comp[0],2)
    
    def MFE(self):
        return self.comp[1]
    
    def AMFE(self):
        return self.comp[1]/len(self.s) * 100
    
    def MFEde(self):
        return 100 * (self.comp[1]- seq.pred_mfe[len(self.s)])/(len(self.s)-8)

def GC(seq):
    return (seq.count("G")+seq.count("C"))/len(seq)* 100

In [ ]:
#MFEde or AMFE may be used // Change code here and below accordingly
MFEde_UTR_5=dict(filter(lambda x: x[0][-2:]==".1",dic.items()))
MFEde_UTR_5={k:seq(v).MFEde() for k,v in dic.items()if 40<=len(v)<=600}

MFEde_UTR_3=dict(filter(lambda x: x[0][-2:]==".1",dic_3.items()))
MFEde_UTR_3={k:seq(v).MFEde() for k,v in dic_3.items()if 40<=len(v)<=600}

### Binning

In [ ]:
#PB-in binning// Length
bin_heat=defaultdict(list)

heat_UTR_5=dict(filter(lambda x: x[0][:-2] in heat_enriched,MFEde_UTR_5.items()))
heat_lengths_5={k: len(v) for k,v in dic.items() if k in MFEde_UTR_5.keys() and k[:-2] in heat_enriched}

for i in range(100,700,100):
    bin_heat[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: x[1]<=100 if i==100 else i-100<x[1]<=i,heat_lengths_5.items())))

    


bin_heat_3=defaultdict(list)

heat_UTR_3=dict(filter(lambda x: x[0][:-2] in heat_enriched,MFEde_UTR_3.items()))
heat_lengths_3={k: len(v) for k,v in dic_3.items() if k in MFEde_UTR_3.keys() and k[:-2] in heat_enriched}

for i in range(100,700,100):
    bin_heat_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: x[1]<=100 if i==100 else i-100<x[1]<=i,heat_lengths_3.items())))    

In [ ]:
Transcriptome binning// Length

UTR5_lengths={k: len(v) for k,v in dic.items() if k in MFEde_UTR_5.keys()}
UTR3_lengths={k: len(v) for k,v in dic_3.items() if k in MFEde_UTR_3.keys()}

bins=defaultdict(list)

for i in range(100,700,100):
    bins[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: x[1]<=100 if i==100 else i-100<x[1]<=i,UTR5_lengths.items())))
    
bins_3=defaultdict(list)

for i in range(100,700,100):
    bins_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: x[1]<=100 if i==100 else i-100<x[1]<=i,UTR3_lengths.items())))

In [ ]:
#PB-in binning// GC%
bin_heat=defaultdict(list)

heat_UTR_5=dict(filter(lambda x: x[0][:-2] in heat_enriched, MFEde_UTR_5.items()))
heat_GC_5={k: GC(v) for k,v in dic.items() if k in MFEde_UTR_5.keys() and k[:-2] in heat_enriched}

for i in range(30,50,5):
    if i==30:
        bin_heat[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: x[1]<=i,heat_GC_5.items())))
        continue
    if i==45:
        bin_heat[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: x[1]>i-5,heat_GC_5.items())))
        continue
    
        
    bin_heat[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: i-5<x[1]<=i,heat_GC_5.items())))


    
bin_heat_3=defaultdict(list)

heat_UTR_3=dict(filter(lambda x: x[0][:-2] in heat_enriched,MFEde_UTR_3.items()))
heat_GC_3={k: GC(v) for k,v in dic_3.items() if k in MFEde_UTR_3.keys() and k[:-2] in heat_enriched}

for i in range(30,50,5):
    if i==30:
        bin_heat_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: x[1]<=i,heat_GC_3.items())))
        continue
    if i==45:
        bin_heat_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: x[1]>i-5,heat_GC_3.items())))
        continue
    
        
    bin_heat_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: i-5<x[1]<=i,heat_GC_3.items())))

In [ ]:
#Transcriptome binning// GC%
UTR5_GC={k: GC(v) for k,v in dic.items() if k in MFEde_UTR_5.keys()}
UTR3_GC={k: GC(v) for k,v in dic_3.items() if k in MFEde_UTR_3.keys()}

bins=defaultdict(list)


for i in range(30,50,5):
    if i==30:
        bins[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: x[1]<=i,UTR5_GC.items())))
        continue
    if i==45:
        bins[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: x[1]>i-5,UTR5_GC.items())))
        continue
    
        
    bins[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: i-5<x[1]<=i,UTR5_GC.items())))
    
    
    
bins_3=defaultdict(list)


for i in range(30,50,5):
    if i==30:
        bins_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: x[1]<=i,UTR3_GC.items())))
        continue
    if i==45:
        bins_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: x[1]>i-5,UTR3_GC.items())))
        continue
    
        
    bins_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: i-5<x[1]<=i,UTR3_GC.items())))

In [ ]:
#PB-in binning// Half Life
from math import isnan
bin_heat=defaultdict(list)

heat_UTR_5=dict(filter(lambda x: x[0][:-2] in ,MFEde_UTR_5.items()))
heat_HL_5={k: Half_Life[k[:-2]] if k[:-2] in Half_Life.keys() else np.nan for k,v in dic.items() if k in MFEde_UTR_5.keys() and k[:-2] in  heat_enriched}

heat_HL_5={k:v for k,v in heat_HL_5.items() if not isnan(v)}
heat_UTR_5={k:v for k,v in heat_UTR_5.items() if k in heat_HL_5.keys()}

for i in np.logspace(2,5,num=4,endpoint=True,base=10):
    if i==100:
        bin_heat[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: x[1]<=i,heat_HL_5.items())))
        continue
    if i==100_000:
        bin_heat[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: x[1]>i/10,heat_HL_5.items())))
        continue
    
        
    bin_heat[f"{i}"]=list(map(lambda x: MFEde_UTR_5[x[0]],filter(lambda x: i/10<x[1]<=i,heat_HL_5.items())))
    
    
    
    
bin_heat_3=defaultdict(list)

heat_UTR_3=dict(filter(lambda x: x[0][:-2] in ,MFEde_UTR_3.items()))
heat_HL_3={k: Half_Life[k[:-2]] if k[:-2] in Half_Life.keys() else np.nan for k,v in dic_3.items() if k in MFEde_UTR_3.keys() and k[:-2] in  heat_enriched}

heat_HL_3={k:v for k,v in heat_HL_3.items() if not isnan(v)}
heat_UTR_3={k:v for k,v in heat_UTR_3.items() if k in heat_HL_3.keys()}

for i in np.logspace(2,5,num=4,endpoint=True,base=10):
    if i==100:
        bin_heat_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: x[1]<=i,heat_HL_3.items())))
        continue
    if i==100_000:
        bin_heat_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: x[1]>i/10,heat_HL_3.items())))
        continue
    
        
    bin_heat_3[f"{i}"]=list(map(lambda x: MFEde_UTR_3[x[0]],filter(lambda x: i/10<x[1]<=i,heat_HL_3.items())))

In [ ]:
#Initiation of UTR Half Lives for transcriptome

UTR5_HL={k: Half_Life[k[:-2]] if k[:-2] in Half_Life.keys() else np.nan for k,v in dic.items() if k in MFEde_UTR_5.keys()}
UTR3_HL={k: Half_Life[k[:-2]] if k[:-2] in Half_Life.keys() else np.nan for k,v in dic_3.items() if k in MFEde_UTR_3.keys()}

UTR5_HL={k:v for k,v in UTR5_HL.items() if not isnan(v)}
UTR3_HL={k:v for k,v in UTR3_HL.items() if not isnan(v)}

filtered_MFEde_UTR_5={k:v for k,v in MFEde_UTR_5.items() if k in UTR5_HL.keys()}
filtered_MFEde_UTR_3={k:v for k,v in MFEde_UTR_3.items() if k in UTR3_HL.keys()}

In [ ]:
#Transcriptome binning// Half Life
bins=defaultdict(list)


for i in np.logspace(2,5,num=4,endpoint=True,base=10):
    if i==100:
        bins[f"{i}"]=list(map(lambda x: filtered_MFEde_UTR_5[x[0]+".1"],filter(lambda x: x[1]<=i and x[0]+".1" in filtered_MFEde_UTR_5.keys(),Half_Life.items())))
        continue
    if i==100_000:
        bins[f"{i}"]=list(map(lambda x: filtered_MFEde_UTR_5[x[0]+".1"],filter(lambda x: x[1]>i/10 and x[0]+".1" in filtered_MFEde_UTR_5.keys(),Half_Life.items())))
        continue
    
        
    bins[f"{i}"]=list(map(lambda x: filtered_MFEde_UTR_5[x[0]+".1"],filter(lambda x: i/10<x[1]<=i and x[0]+".1" in filtered_MFEde_UTR_5.keys(),Half_Life.items())))
    
    
    
bins_3=defaultdict(list)


for i in np.logspace(2,5,num=4,endpoint=True,base=10):
    if i==100:
        bins_3[f"{i}"]=list(map(lambda x: filtered_MFEde_UTR_3[x[0]+".1"],filter(lambda x: x[1]<=i and x[0]+".1" in filtered_MFEde_UTR_3.keys(),Half_Life.items())))
        continue
    if i==100_000:
        bins_3[f"{i}"]=list(map(lambda x: filtered_MFEde_UTR_3[x[0]+".1"],filter(lambda x: x[1]>i/10 and x[0]+".1" in filtered_MFEde_UTR_3.keys(),Half_Life.items())))
        continue
    
        
    bins_3[f"{i}"]=list(map(lambda x: filtered_MFEde_UTR_3[x[0]+".1"],filter(lambda x: i/10<x[1]<=i and x[0]+".1" in filtered_MFEde_UTR_3.keys(),Half_Life.items())))

In [ ]:
medians_heat=[np.median(i) for i in bin_heat.values()]
medians_heat_3=[np.median(i) for i in bin_heat_3.values()]

medians=[np.median(i) for i in bins.values()]
medians_3=[np.median(i) for i in bins_3.values()]

### Plotting

In [ ]:
%matplotlib notebook

fig=plt.figure(figsize=(20,5))
gs = gridspec.GridSpec(2,2, height_ratios=[3, 1])

width=0.2

ax0=plt.subplot(gs[0,0])
ax0.scatter(x=list(UTR5_lengths.values()),y=list(MFEde_UTR_5.values()),
            marker="1",color="#021012")
ax0.scatter(x=list(UTR3_lengths.values()),y=list(MFEde_UTR_3.values()),
            marker="2",color="#a6b7a8")

ax0.set_ylim([-40,40]) #MFEde
#ax0.set_ylim([-0.6,0]) #AMFE
#ax0.set_xlim([5,75]) #GC%
#ax0.set_xscale("log") #Half Life
#ax0.set_xticks(xti) #Half Life
ax0.set_yticks(np.arange(-40,50,20)) #MFEde
ax0.set_ylabel("MFEde (kcal/mol)")
ax0.legend(labels=["5'UTR","3'UTR"],loc=4)

ax0.set_title("Transcriptome")

x=np.arange(len(medians))

ax1=plt.subplot(gs[1,0])
ax1.bar(x-width/2,medians,width,color="#021012")
ax1.bar(x+width/2,medians_3,width,color="#a6b7a8")

ax1.plot(x-width/2,medians,color="#021012")
ax1.plot(x+width/2,medians_3,color="#a6b7a8")

ax1.set_xticks(ax1.get_xticks(),[" ","<100","100","200","300","400","500"," "]) #Length
#ax1.set_xticks(ax1.get_xticks(),["","<30","","30-35","","35-40","","40",""]) #GC%
#ax1.set_xticks(ax1.get_xticks(),["","<100","","100-1000","","1000-10000","","<10000",""]) #Half Life
ax1.set_xlim([-0.9,5.5]) #Length
#ax1.set_xlim([-0.4,3.4]) #GC%, Half Life
ax1.set_ylim([0,20]) #MFEde
#ax1.set_yticks([0,-0.2]) #AMFE
#ax1.invert_yaxis() #AMFE

ax1.set_xlabel("GC%")

ax3=plt.subplot(gs[0,1])
ax3.scatter(x=list(heat_lengths_5.values()),y=list(heat_UTR_5.values()),
            marker="1",color="#021012")
ax3.scatter(x=list(heat_lengths_3.values()),y=list(heat_UTR_3.values()),
            marker="2",color="#a6b7a8")

ax3.set_ylim([-40,40]) #MFEde
#ax3.set_ylim([-0.6,0]) #AMFE
#ax3.set_xlim([5,75]) #GC%
#ax3.set_xticks(list(range(0,5000,1000))) #Length AMFE
#ax3.set_xscale('log') #Half Life
#ax3.set_xticks(xti) #Half Life
ax3.set_yticks(np.arange(-40,50,20)) #MFEde
ax3.set_ylabel("MFEde (kcal/mol)")
ax3.legend(labels=["5'UTR","3'UTR"],loc=4)
ax3.set_title("Heat enriched")

ax4=plt.subplot(gs[1,1])
ax4.bar(x-width/2,medians_heat,width,color="#021012")
ax4.bar(x+width/2,medians_heat_3,width,color="#a6b7a8")

ax4.plot(x-width/2,medians_heat,color="#021012")
ax4.plot(x+width/2,medians_heat_3,color="#a6b7a8")

ax4.set_xlim([-0.2,3.4]) #GC%
ax4.set_xticks(ax1.get_xticks(),["","<100","100","200","300","400","500",""]) #Length
#ax4.set_xticks(ax1.get_xticks(),["","<30","","30-35","","35-40","","40",""]) #GC%
#ax4.set_xticks(ax1.get_xticks(),["","<100","","100-1000","","1000-10000","","<10000",""]) #Half Life
ax4.set_xlim([-0.9,5.5]) #Length
ax4.set_ylim([0,20]) #MFEde
#ax4.invert_yaxis() #AMFE
#ax4.set_yticks([0,-0.2]) #AMFE
ax4.set_xlabel("GC%")

sns.despine()

#
comparisons=[
#["MFEde","GC","5'UTR","Transcriptome",spearmanr(list(MFEde_UTR_5.values()),list(UTR5_GC.values()))],  
#["MFEde","GC","3'UTR","Transcriptome",spearmanr(list(MFEde_UTR_3.values()),list(UTR3_GC.values()))],
#["MFEde","GC","5'UTR","heat enriched",spearmanr(list(heat_UTR_5.values()),list(heat_GC_5.values()))],
#["MFEde","GC","3'UTR","heat enriched",spearmanr(list(heat_UTR_3.values()),list(heat_GC_3.values()))],
]


#plt.savefig("name.tiff",dpi=600)